<a href="https://colab.research.google.com/github/wrwills/pml/blob/master/course/Problem034_recs_playground/034.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's have a play

In [4]:
!pip install surprise

     |████████████████████████████████| 11.8MB 5.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618266 sha256=833f44de3cc1ba70ca9e5f9233e602b9edd2e535eba6dfb1ecb4f2f2e293bd46
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [6]:
from surprise import SVD
from surprise import Dataset

from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy

import csv

data = Dataset.load_builtin('ml-100k')

def get_movies():
    movies = {}
    with open(data.ratings_file.replace("u.data","u.item"), 'r', encoding = "ISO-8859-1") as csvfile:
        moviesreader = csv.reader(csvfile, delimiter='|')
        for row in moviesreader:
            movies[int(row[0])] = row[1]
    return movies

def get_recs(algo,uid):
    recs = []
    for item in algo.trainset.all_items():
        pred = algo.predict(str(uid), str(item))
        recs.append((pred.est,item))
    recs.sort(reverse = True)
    return recs

def top_recs(i,recs):
    return list(map(lambda x: (x[0],movies[x[1]]), recs[:10]))

def get_ratings_by_user(uid):
    ratings = list(map(lambda x: tuple(reversed(x)), trainset.ur[uid]))
    ratings.sort(reverse = True)
    return ratings

def show_movie_ratings(ratings):
    return list(map(lambda x: (x[0],x[1],movies.get(x[1],x[1])), ratings))

In [9]:
movies = get_movies()
trainset, testset = train_test_split(data, test_size=.15)

In [10]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
#algo = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})

algo.fit(trainset)
# now we can test predictions from the model against our testset
predictions = algo.test(testset)

# and we can test against test data
# mse -- mean sqare error
# rmse  -- root mean square error
# - how close fitted line is to data points

accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9420


0.9420274382573778

In [12]:
algo = SVD(n_factors = 20)
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)


RMSE: 0.9414


0.9414295071526064